# syft-widget tutorial: resilient, graceful, file-backed widgets

Welcome to syft-widget! The syft ecosystem enables apps/AI/queries across distributed state, where each part of that state is stored and controlled privately by people on the network. This is the private data internet.

However, it can be difficult to work with distributed state for data (i.e. files) you can't see. So, the syft ecosystem relies on powerful, file-backed widgets to help users see that state in a live, intuitive way.

However, jupyter doesn't really like widgets/UIs, and it ESPECIALLY doesn't like widgets/UIs which need to live-update against the filesystem. Syft-widget is here to help.

1) **UIs are local servers**: Syft-serve enables all UIs to run in syft-serve servers
2) **UIs spin up instantly**: all widgets load in a snappy fashion because they launch with a snapshot of the API endpoints (checkpoint of data), and then roll over to the live server backends as soon as they launch.
3) **UIs run in iframes**: Because Jupyter has such a complex DOM, it can be really hard to develop html/javascript/css in jupyter, so we launch everything in iframes to protect the widget DOM from jupyer.

But enough with all that — let's just build some cool stuff.

## Part 1: Create your first widget

In [38]:
import syft_widget as sw

In [48]:
class LiveClock(sw.DynamicWidget):

    # create endpoints for website to use
    def get_endpoints(self):
        
        @self.endpoint("/api/time")
        def get_time():
            import time
            return {"time": time.strftime("%H:%M:%S")}

    # script which gets live-updated based on endpoints every 
    # 1 seconds (without reloading the DOM)
    def get_template(self):
        return '''
            <h1>Time:<span data-field="time">{time}</span></h1>
        '''
        
clock = LiveClock("Live Clock", height="50px")
clock

# Part 2: See the backend

In [49]:
import syft_serve as ss

In [50]:
ss.servers

Name,Port,Status,Endpoints,Uptime,Expires In,PID
live_clock,8001,✅ Running,/api/time,23m 5s,23h 36m,16737


In [51]:
ss.servers['live_clock']

Status:,✅ Running
URL:,http://localhost:8001
Endpoints:,/api/time
Uptime:,23m 6s
Expires In:,⏰ 23h 36m
PID:,16737


# Part 3: Launch the same widget again

In [56]:
clock = LiveClock("Live Clock", height="55px")
clock

In [ ]:
# notice how it does NOT create a new server... cuz the name

In [53]:
ss.servers

Name,Port,Status,Endpoints,Uptime,Expires In,PID
live_clock,8001,✅ Running,/api/time,23m 38s,23h 36m,16737


# Part 4: Launch with a new name

In [57]:
clock = LiveClock("Live Clock 2", height="55px")
clock

# Part 5: Break the widget to see how it works

In [71]:
class LiveClockBroken(sw.DynamicWidget):

    # create endpoints for website to use
    def get_endpoints(self):
        
        @self.endpoint("/api/time")
        def get_time():
            import time
            return {"time": time.strftime("%H:%M:%S")}

    # script which gets live-updated based on endpoints every 
    # 1 seconds (without reloading the DOM)
    def get_template(self):
        return '''
            <h1>Time:<span data-field="time">{JWEIJGEIWJWEIGJWEIGJWEG}</span></h1>
        '''
        
clock = LiveClockBroken("Live Clock", height="60px")
clock

### How it works:

- **Initial Load:** before the server is ready, the python looks for {} and replaces variables in those brackets with outputs from the endpoints... but this comes from the local python runtime.
- **Server Query:** after the server is ready, the javascript finds that server and then looks for *span* tags which associate with the right keyword (i.e. "time"). The javascript then replaces the text in those span tags with whatever the endpoigns say... once a second. This means the DOM of the widgets doesn't reload (e.g. for smooth scrolling, animations, etc.)
- **Reload:** notice how the flicker still happens when you reload, but keep in mind it is NOT launching a server every time you load the widget... as long as you don't change the name. (notice how this widget is still using the Live Clock API even though it's being wrapped in a new LiveClockBroken class).

Consequently, LiveClockBroken briefly flashes "{JWEIJGEIWJWEIGJWEIGJWEG}" before it talks to the server, because we don't have a variable called JWEIJGEIWJWEIGJWEIGJWEG, but we do have a data-field called "time" so the surrounding SPAN tag kicks in after 1 second and replaces {JWEIJGEIWJWEIGJWEIGJWEG}.

So you get the INSTANT snappiness of immediately delivered HTML, with the full power and flexibility of a backup server which can live-query various data sources (top priority: the ~/SyftBox filesystem).

# Part 6: A widget with python dependencies

In [74]:
class SystemMonitor(sw.DynamicWidget):
    def get_endpoints(self):
        @self.endpoint("/api/system")
        def get_system_info():
            import psutil
            from datetime import datetime
            return {
                "cpu_percent": psutil.cpu_percent(interval=0.1),
                "memory_percent": psutil.virtual_memory().percent,
                "disk_percent": psutil.disk_usage('/').percent,
                "timestamp": datetime.now().strftime("%H:%M:%S")
            }
    
    def get_template(self):
        return '''
        <div style="font-family: Arial; padding: 20px; background: #f0f0f0; border-radius: 8px;">
            <h3>System Monitor</h3>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px;">
                <div>
                    <strong>CPU:</strong> <span data-field="cpu_percent">{cpu_percent}</span>%
                </div>
                <div>
                    <strong>Memory:</strong> <span data-field="memory_percent">{memory_percent}</span>%
                </div>
                <div>
                    <strong>Disk:</strong> <span data-field="disk_percent">{disk_percent}</span>%
                </div>
                <div>
                    <strong>Updated:</strong> <span data-field="timestamp">{timestamp}</span>
                </div>
            </div>
        </div>
        '''

monitor = SystemMonitor("System Monitor", 
                        dependencies=['psutil'],
                        height="180px")
monitor

# Part 7: Debug a broken widget

As you develop widgets, you'll break stuff. You can fix it using some tools.

In [96]:
class AnotherBrokenClock(sw.DynamicWidget):

    # create endpoints for website to use
    def get_endpoints(self):
        
        @self.endpoint("/api/time")
        def get_time():
            import time
            print(THIS_VARIABLE_DOESNT_EXIST)
            return {"time2": time.strftime("%H:%M:%S")}

    # script which gets live-updated based on endpoints every 
    # 1 seconds (without reloading the DOM)
    def get_template(self):
        return '''
            <h1>Time:<span data-field="time2">{time2}</span></h1>
        '''
        
clock = AnotherBrokenClock("Broken Clock", height="50px")
clock

### How did it break?

It broke because the get_time() method is printing a variable that doesn't exist

1) The clock still renders because the html is still html
2) The clock doesn't show the time because the get_time() method breaks upon initial display
3) The clock doesn't update because teh server is ALSO broken in the same way. 

In [97]:
clock.server

Status:,✅ Running
URL:,http://localhost:8005
Endpoints:,/api/time
Uptime:,3m 3s
Expires In:,⏰ 23h 56m
PID:,21485


In [101]:
# we can even see the full stack trace
print(clock.server.stderr.tail(20))

                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/atrask/.syft_logs/server_envs/broken_clock/.venv/lib/python3.12/site-packages/fastapi/routing.py", line 215, in run_endpoint_function
    return await run_in_threadpool(dependant.call, **values)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/atrask/.syft_logs/server_envs/broken_clock/.venv/lib/python3.12/site-packages/starlette/concurrency.py", line 38, in run_in_threadpool
    return await anyio.to_thread.run_sync(func)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/atrask/.syft_logs/server_envs/broken_clock/.venv/lib/python3.12/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/atrask/.syft_logs/server_envs/broken_clock/.venv/lib/python3.12/site-packages/anyio/_backends/_asyncio.py", line 2470, in run_sync_in_worker_thread
    retur

### How do we fix it?

We just recreate the class 

In [106]:
clock.server.terminate()

In [108]:
class FixedClock(sw.DynamicWidget):

    # create endpoints for website to use
    def get_endpoints(self):
        
        @self.endpoint("/api/time2")
        def get_time():
            import time
            # print(THIS_VARIABLE_DOESNT_EXIST)
            return {"time2": time.strftime("%H:%M:%S")}

    # script which gets live-updated based on endpoints every 
    # 1 seconds (without reloading the DOM)
    def get_template(self):
        return '''
            <h1>Time:<span data-field="time2">{time2}</span></h1>
        '''
        
clock = FixedClock("Fixed Clock", height="60px")
clock

# Part 8: Dark Mode in Jupyter/Colab

Now let's create a clock which renders differently in dark/light mode

In [110]:
import syft_widget as sw

class DarkClock(sw.DynamicWidget):

  def get_endpoints(self):
      @self.endpoint("/api/time")
      def get_time():
          import time
          return {"time": time.strftime("%H:%M:%S")}

  def _get_widget_styles(self):
      """Base widget styles"""
      return """
      .clock-container {
          display: flex;
          align-items: center;
          justify-content: center;
          padding: 15px;
          border-radius: 12px;
          font-family: 'Monaco', 'Menlo', 'Ubuntu Mono', monospace;
          font-size: 24px;
          font-weight: bold;
          transition: all 0.3s ease;
          box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
      }
      .clock-label {
          margin-right: 10px;
      }
      .clock-time {
          font-variant-numeric: tabular-nums;
          letter-spacing: 1px;
      }
      """

  def _get_css_light(self):
      """Light theme styles"""
      return """
      .clock-container {
          background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
          color: white;
          border: 2px solid #e2e8f0;
      }
      body {
          background-color: #f8fafc;
      }
      """

  def _get_css_dark(self):
      """Dark theme styles"""
      return """
      .clock-container {
          background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
          color: #00d4ff;
          border: 2px solid #4a5568;
          box-shadow: 0 4px 8px rgba(0, 212, 255, 0.2);
      }
      body {
          background-color: #0f172a;
      }
      .clock-time {
          text-shadow: 0 0 10px rgba(0, 212, 255, 0.5);
      }
      """

  def get_template(self):
      return '''
      <div class="clock-container">
          <span class="clock-label">Time:</span>
          <span class="clock-time" data-field="time">{time}</span>
      </div>
      '''

clock = DarkClock("Dark Clock", height=80)
clock

## 9. Server Expiration and Auto-Cleanup 🆕

syft-serve now includes automatic server expiration to prevent server accumulation! By default, all servers expire after 24 hours.

In [3]:
import syft_widget as sw

In [16]:
class ShortLivedClock(sw.DynamicWidget):

    # create endpoints for website to use
    def get_endpoints(self):
        
        @self.endpoint("/api/short_time")
        def get_time():
            import time
            return {"short_time": time.strftime("%H:%M:%S")}

    # script which gets live-updated based on endpoints every 
    # 1 seconds (without reloading the DOM)
    def get_template(self):
        return '''
            <h1>Time:<span data-field="short_time">{short_time}</span></h1>
        '''

# note that in practice it takes 20-30 seconds for the server to actually die
clock = ShortLivedClock("Short Lived Clock", 
                        expiration_seconds=10,
                        height="50px")
clock

### How Server Expiration Works

1. **Default Behavior**: All servers expire after 24 hours (86400 seconds)
2. **Custom Duration**: Set `expiration_seconds` to any positive value
3. **Never Expire**: Set `expiration_seconds=-1` for permanent servers
4. **Self-Destruct**: Servers automatically terminate when expired
5. **Background Monitoring**: Expiration is checked every minute

This feature prevents server accumulation from forgotten notebook sessions!

## 10. Cleanup and Server Management

When you're done, you can stop servers and clean up:

In [22]:
import syft_serve as ss
ss.servers.terminate_all()

{'tracked_total': 0,
 'tracked_terminated': 0,
 'tracked_failed': [],
 'orphaned_discovered': 0,
 'orphaned_terminated': 0,
 'orphaned_failed': [],
 'success': True}

# Next Steps

Now you've learned how to create powerful widgets! Continue to the next syft tutorial to see some pre-made widets in actoin